In [2]:
import psycopg2 as pg
from psycopg2.extras import execute_values
import pandas as pd
import requests
import zipfile
import io

from config import DATABASE_URI, FIRSTRATE_URL

In [6]:
conn = pg.connect(DATABASE_URI)
with conn:
    with conn.cursor() as cur:
        cur.execute(
            "SELECT MAX(date) FROM prices WHERE (frequency='MINUTE' AND ticker='^GSPC');"
        )
        date = cur.fetchall()[0][0]

r = requests.get(FIRSTRATE_URL, stream=True)
file = io.BytesIO(r.raw.read())
file = zipfile.ZipFile(file)

spx = pd.read_csv(
    file.open("SPX_1min.txt"),
    names=["date", "open", "high", "low", "close", "?"],
    parse_dates=["date"],
)
# Get only new records for insertion
spx = spx[spx["date"] > date]
# Firstrate includes a column thats always zero for some reason
spx = spx.drop(columns=["?"])
# Set database columns
spx["frequency"] = "MINUTE"
spx["ticker"] = "^GSPC"

In [8]:
sql = f"""INSERT INTO prices ({", ".join(spx.columns)}) VALUES %s ON CONFLICT (ticker, date, frequency) DO UPDATE SET ({", ".join(spx.columns)}) = ({", ".join("EXCLUDED." + spx.columns)})"""

with pg.connect(DATABASE_URI) as conn:
    with conn.cursor() as cur:
        execute_values(cur, sql, list(spx.itertuples(index=False, name=None)))

In [16]:
conn = pg.connect(DATABASE_URI)
with conn:
    with conn.cursor() as cur:
        cur.execute(
            "SELECT MAX(date) FROM prices WHERE (frequency='MINUTE' AND ticker='^GSPC');"
        )
        date = cur.fetchall()[0][0]

r = requests.get(FIRSTRATE_URL, stream=True)
file = io.BytesIO(r.raw.read())
file = zipfile.ZipFile(file)

spx = pd.read_csv(
    file.open("SPX_1min.txt"),
    names=["date", "open", "high", "low", "close", "?"],
    parse_dates=["date"],
)

In [18]:
spx = pd.read_csv(
    file.open("SPX_1hour.txt"),
    names=["date", "open", "high", "low", "close", "?"],
    parse_dates=["date"],
)

In [19]:
spx

,date,open,high,low,close,?
0,2007-04-30 09:00:00,1494.07,1495.36,1491.92,1493.42,NaN
1,2007-04-30 10:00:00,1493.91,1497.16,1492.17,1497.00,NaN
2,2007-04-30 11:00:00,1496.89,1496.93,1493.63,1494.00,NaN
3,2007-04-30 12:00:00,1493.99,1496.62,1493.73,1495.29,NaN
4,2007-04-30 13:00:00,1495.01,1496.83,1494.48,1495.09,NaN
...,...,...,...,...,...,...
28723,2021-08-13 12:00:00,4464.20,4465.38,4462.92,4464.87,NaN
28724,2021-08-13 13:00:00,4464.88,4467.03,4463.50,4466.37,NaN
28725,2021-08-13 14:00:00,4466.39,4466.80,4461.19,4463.74,NaN
28726,2021-08-13 15:00:00,4463.72,4468.08,4462.70,4467.97,NaN
